In [43]:
import cv2
import numpy as np


In [44]:
#Read Image

img = cv2.imread('mel2.png')


In [45]:
#Display Image

cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [46]:
#resized image 

resized_image = cv2.resize(img, (300,300)) 

In [47]:
#Display Image

cv2.imshow('redimensionada',resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
# apply the 3x3 median filter on the image

processed_image = cv2.medianBlur(resized_image, 7)

# display image

cv2.imshow('Median Filter Processing', processed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [55]:
gray = cv2.cvtColor(processed_image,cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
cv2.imshow('Binarização',mask)
cv2.waitKey(0)

-1

In [56]:
#Preenche os quatro cantos da imagem binária

w, h = mask.shape[::-1]
cv2.floodFill(mask, None, (0, 0), 0)
cv2.floodFill(mask, None, (w-1, 0), 0)
cv2.floodFill(mask, None, (0, h-1), 0)
cv2.floodFill(mask, None, (w-1, h-1), 0)
cv2.imshow('mask',mask)
cv2.waitKey(0)

#Lógica AND para obter da imagem original a encontrada pela criação do mask

img = cv2.bitwise_and(processed_image, processed_image, mask=mask )
cv2.imshow('AND',img)
cv2.waitKey(0)

-1

In [57]:
kernel = np.ones((3,3),np.uint8)

#Canny Edges

edges = cv2.Canny(img, 100,200)
dilate = cv2.dilate(edges,kernel,iterations=1)
dilate = cv2.bitwise_not(dilate)
cv2.imshow('Canny',dilate)
cv2.waitKey(0)


-1

In [58]:
#Lógica OR para retirar da imagem original os pêlos encontrados

img = cv2.bitwise_or(img, img, mask=dilate )
cv2.imshow('Edges',img)
cv2.waitKey(0)

-1

In [59]:
#Interpolação da imagem para preencher os vazios

dilate = cv2.bitwise_not(dilate)
inpaint = cv2.inpaint(img, dilate, 3,cv2.INPAINT_TELEA)
cv2.imshow('InPaint', inpaint)
cv2.waitKey(0)

-1